# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Import collected city weather as DataFrame
city_weather = pd.read_csv("../WeatherPy/Resources/Sample_Weather_Data.csv")
# Clean cities with no weather data
city_weather_clean = city_weather.dropna()
city_weather_clean.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,"kruisfontein, za",-34.0033,24.7314,59.00,77.0,42.0,3.00,ZA,1.612214e+09
2,"slave lake, ca",55.2834,-114.7690,14.00,92.0,90.0,11.50,CA,1.612214e+09
3,"puerto ayora, ec",-0.7393,-90.3518,84.20,65.0,0.0,12.66,EC,1.612214e+09
4,"provideniya, ru",64.3833,-173.3000,7.45,91.0,100.0,28.99,RU,1.612214e+09
5,"saskylakh, ru",71.9167,114.0833,-30.32,84.0,100.0,7.76,RU,1.612214e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_clean[["Lat", "Lng"]].astype(float)

# Store associated humidity values for weights
humidity = city_weather_clean["Humidity"].astype(float)


In [7]:
# Customize the size of the map figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create the figure with layout parameters
fig = gmaps.figure(layout=figure_layout)

# Create the heat layer with location data and humidity weights 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

# Add the layer to the figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Refine the weather results to readings that would seem fitting for a vacation
vacation_weather = city_weather_clean.loc[(city_weather_clean["Max Temp"] < 80) &
                                          (city_weather_clean["Max Temp"] > 70) &
                                          (city_weather_clean["Humidity"] < 80) & 
                                          (city_weather_clean["Wind Speed"] < 10) &
                                          (city_weather_clean["Cloudiness"] == 0)]

# Check the count to see if it's a reasonable number
vacation_weather.count()

City          12
Lat           12
Lng           12
Max Temp      12
Humidity      12
Cloudiness    12
Wind Speed    12
Country       12
Date          12
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(columns=['Lat', 'Lng', 'Hotel Name', 'City', 'Country'])
hotel_df

,Lat,Lng,Hotel Name,City,Country


In [11]:
# set default for target coordinates
target_coordinates = ""
# set radius of 5000 meters
target_radius = 5000
# set the type of results for Google to search for
target_type = "lodging"

# Iterate through vacation data for coordinates and create hotel dataframe
for index, row in vacation_weather.iterrows():
    
    # geocoordinates
    lat = vacation_weather.loc[index, "Lat"]
    lng = vacation_weather.loc[index, "Lng"]
    
    # Populate what is already known into the new hotel dataframe
    target_coordinates = f'{lat}, {lng}'
    hotel_df.loc[index, "Lat"] = lat
    hotel_df.loc[index, "Lng"] = lng
    hotel_df.loc[index, "City"] = vacation_weather.loc[index, "City"]
    hotel_df.loc[index, "Country"] = vacation_weather.loc[index, "Country"]
    
    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # base Google Places url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    try:
        # run a request using assembled params dictionary
        response = requests.get(base_url, params=params)

        places_data = response.json()
    
        hotel_df.loc[index, "Hotel Name"] = places_data["results"][0]["name"]
            
    except:
        hotel_df.loc[index, "Hotel Name"] = "Unknown"
        
hotel_df

,Lat,Lng,Hotel Name,City,Country
13,-45.5752,-72.0662,Diego De Almagro Coyhaique,"coihaique, cl",CL
20,11.5221,13.6856,UN Humanitarian Hub,"bama, ng",NG
72,-13.7,-76.2167,Hotel San Isidro Oficial,"pisco, pe",PE
196,14.8961,-24.4956,Tortuga B&B,"sao filipe, cv",CV
276,16.9833,73.3,Hotel Landmark,"ratnagiri, in",IN
356,23.7667,-98.2167,Hotel Chicago,"soto la marina, mx",MX
370,10.5222,7.4383,HAMDALA HOTEL,"kaduna, ng",NG
401,11.2842,49.1816,Red Sea Hotel,"bosaso, so",SO
430,14.1212,15.3103,Unknown,"mao, td",TD
477,-23.85,151.25,Highpoint International,"gladstone, au",AU


In [13]:
# Starter code
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…